In [7]:
import matplotlib
matplotlib.use('agg')
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import nltk.corpus
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt
import os

data_file = 'app/static/data/reviews.csv'

df = pd.read_csv(data_file, encoding='latin1')


# This works.
def review_by_rating(reviews_df):
    reviews_df['rating_desc'] = None
    reviews_df['rating_desc'].loc[reviews_df['rating'] < 3] = 'Low Rating (One or Two)'
    reviews_df['rating_desc'].loc[reviews_df['rating'] > 3] = 'High Rating (Four or Five)'
    reviews_by_rating = reviews_df.groupby('rating_desc')['review'].apply(list)
    reviews_by_rating = reviews_by_rating.reset_index(drop=False)
    return reviews_by_rating

def make_word_cloud(df, ngram_min, ngram_max, name):
    ## Change this back to:os.path.abspath(os.path.dirname(__file__)) + '/static/word_cloud/'
    my_path = os.path.abspath(os.path.dirname(__file__)) + '/static/word_cloud/'
    stopwords = set(nltk.corpus.stopwords.words('english'))
    filenames=[]
    for ind, row in df.iterrows():
        data = row['review']
        num_words = 200
        ngram_range = (ngram_min, ngram_max)

        count_vectorizer = CountVectorizer(lowercase=True,
                                           stop_words=stop_words,
                                           ngram_range=ngram_range)
        counts = count_vectorizer.fit_transform(data)
        counts = counts.toarray().sum(axis=0)
        count_weighting = dict(zip(count_vectorizer.get_feature_names(), counts))
        count_weighting_df = pd.DataFrame.from_dict(count_weighting, orient='index')
        count_weighting_df = count_weighting_df.reset_index(drop=False)
        count_weighting_df.columns = ['word', 'count']

        count_weighting_df = count_weighting_df.sort_values(['count'], ascending=False)
        count_weighting_df = count_weighting_df.set_index('word')

        word_cloud_freq = count_weighting_df['count'].head(num_words).to_dict()
        wordcloud = WordCloud(collocations=False).generate_from_frequencies(word_cloud_freq)
        plotname = '{}_{}.png'.format(name,ind+1)
        filenames.append(plotname)
        url = my_path + plotname
        fig = plt.figure(figsize=(10,10))
        plt.imshow(wordcloud, cmap=plt.cm.bone, interpolation='bilinear')
        plt.axis("off")
        fig.savefig(url,transparent = True, bbox_inches = 'tight', pad_inches = 0)
    return filenames


data_file = 'app/static/data/reviews.csv'


make_word_cloud(df, 2,3,'the')


NameError: name '__file__' is not defined